In [33]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from scrapy import Selector
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [362]:
url="https://defypunk.com/punks"
n=10000 #number cryptopunks
df=pd.DataFrame(columns=['Number','Type','Skin','NumberTraits','Traits','Rarity','HiddenTraits','Owner'])

In [301]:
#Activamos que Selenium esté controlando Chrome y abrimos la web
#Necesitamos descargarnos previamente el driver del navegador que usamos.
DRIVER_PATH = 'Downloads/chromedriver_win32/chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)

In [376]:
for i in range(n):
    url2=url+'/'+str(i)
    driver.implicitly_wait(15)
    driver.get(url2)
    #rarity
    results=driver.find_elements_by_class_name('font-bold')
    rarity=results[0].text.split()[0][:-2].replace(',','')
    # Get Number,Type,Skin,NumberTraits,Traits,HiddenTraits
    results=driver.find_elements_by_class_name('font-semibold')
    data=[i.text for i in results]    
    hidden=[i for i in data if 'Hidden' in i]
    data=[i for i in data if i not in hidden]
    #Number
    data[0]=data[0].split('#')[1]
    #Number Traits
    data[3]=data[3].split('(')[1].rstrip(')')
    #Traits
    data[4]=data[4:4+int(data[3])]
    data=data[:5]
    #Get owner
    results=driver.find_elements_by_class_name('mt-4')
    owner=[i.text for i in results]
    owner='' if 'ERC721' in owner[1] else owner[1].split('(')[0].split()[-1]
    #Get HiddenTraits
    hidden='' if len(hidden)==0 else hidden
    
    data.append(rarity)
    data.append(hidden)
    data.append(owner)
    
    df.loc[len(df)]=data
    if i%1000==0:
        print('%i/10000'%(i+1))

1/10000
1001/10000
2001/10000
3001/10000
4001/10000
5001/10000
6001/10000
7001/10000
8001/10000
9001/10000


In [389]:
#Guardamos en csv
df.to_csv('df_crypto.csv', index=False)

In [391]:
df=pd.read_csv('df_crypto.csv')

In [392]:
df.head()

,Number,Type,Skin,NumberTraits,Traits,Rarity,HiddenTraits,Owner
0,0,Female,Mid-skinned,3,"['Blonde Bob', 'Earring', 'Green Eye Shadow']",2023,['Hidden Earring'],0xE08c32
1,1,Male,Dark-skinned,2,"['Mohawk', 'Smile']",4352,NaN,0xB88F61
2,2,Female,Light-skinned,1,['Wild Hair'],8090,NaN,0x897aEA
3,3,Male,Dark-skinned,3,"['Nerd Glasses', 'Pipe', 'Wild Hair']",7896,NaN,0xC352B5
4,4,Male,Mid-skinned,4,"['Big Shades', 'Earring', 'Goat', 'Wild Hair']",7332,NaN,0xC352B5


In [393]:
df.tail()

,Number,Type,Skin,NumberTraits,Traits,Rarity,HiddenTraits,Owner
9995,9995,Female,Albino-skinned,2,"['Purple Eye Shadow', 'Straight Hair Dark']",2466,NaN,0x758353
9996,9996,Male,Light-skinned,4,"['Cigarette', 'Crazy Hair', 'Earring', 'Smile']",4323,NaN,0xA9bFA9
9997,9997,Zombie,Zombie-skinned,2,"['Cap Forward', 'Front Beard']",498,NaN,0x279679
9998,9998,Female,Mid-skinned,3,"['Black Lipstick', 'Clown Eyes Green', 'Wild W...",1271,NaN,0xcFD9cc
9999,9999,Female,Dark-skinned,2,"['Mohawk', 'Nerd Glasses']",9233,NaN,0x0A383e


In [394]:
df.NumberTraits.value_counts()

3    4501
2    3560
4    1420
1     333
5     166
6      11
0       8
7       1
Name: NumberTraits, dtype: int64

In [426]:
df.Type.value_counts()

Male      6039
Female    3840
Zombie      88
Ape         24
Alien        9
Name: Type, dtype: int64

In [427]:
df.HiddenTraits.value_counts()

['Hidden Earring']                                      478
['Hidden Frown']                                         44
['Hidden Mole']                                          41
['Hidden Smile']                                         32
['Hidden Lipstick']                                      18
['Hidden Handlebars']                                     5
['Hidden Mustache']                                       3
['Hidden Earring', 'Hidden Lipstick']                     3
['Hidden Buck Teeth']                                     2
['Hidden Earring', 'Hidden Mole', 'Hidden Lipstick']      1
['Hidden Earring', 'Hidden Mole']                         1
['Hidden Mole', 'Hidden Frown']                           1
Name: HiddenTraits, dtype: int64

In [428]:
df.Skin.value_counts()

Mid-skinned       3031
Light-skinned     3006
Dark-skinned      2824
Albino-skinned    1018
Zombie-skinned      88
Ape-skinned         24
Alien-skinned        9
Name: Skin, dtype: int64

## Transacciones

In [550]:
df_trans2=pd.DataFrame(columns=['Type','From','To','Amount','Transaction','Number'])
for i in range(n):
    df_trans=pd.DataFrame(columns=['Type','From','To','Amount','Transaction'])
    url2=url+'/'+str(i)
    driver.implicitly_wait(15)
    driver.get(url2)    
    results2=driver.find_elements_by_tag_name('table')
    trans=[i.text for i in results2[0].find_elements_by_tag_name('td')]
    length=len(trans)//len(df_trans.columns)
    for i in range(length):
        df_trans.loc[len(df_trans)]=trans[0:5]
        trans=trans[5:]
    results=driver.find_elements_by_class_name('font-semibold')
    data=[i.text for i in results]
    #Number
    data=data[0].split('#')[1]
    df_trans['Number']=data    
    df_trans2=pd.concat([df_trans2, df_trans], axis=0)    

KeyboardInterrupt: 

In [551]:
#Guardamos en csv
df_trans2.to_csv('df_trans.csv', index=False)

In [560]:
prueba=pd.read_csv('df_trans.csv')

In [561]:
prueba.head()

,Type,From,To,Amount,Transaction,Number
0,Bid withdrawn,0x983ace,NaN,250.0Ξ,"May 10, 2021 0x0987ef",0
1,Bid,0x983ace,NaN,250.0Ξ,"Apr 11, 2021 0xdbf1a0",0
2,Bid withdrawn,0xd7510a,NaN,100.0Ξ,"Mar 3, 2021 0x580c05",0
3,Bid,0xd7510a,NaN,100.0Ξ,"Feb 21, 2021 0x5377c3",0
4,Bid withdrawn,0x63a9db,NaN,69.0Ξ,"Sep 16, 2020 0xbce1fe",0


In [562]:
prueba['TransactionNumber']=prueba['Transaction'].str.split().apply(lambda x: x[-1])

In [563]:
prueba.head()

,Type,From,To,Amount,Transaction,Number,TransactionNumber
0,Bid withdrawn,0x983ace,NaN,250.0Ξ,"May 10, 2021 0x0987ef",0,0x0987ef
1,Bid,0x983ace,NaN,250.0Ξ,"Apr 11, 2021 0xdbf1a0",0,0xdbf1a0
2,Bid withdrawn,0xd7510a,NaN,100.0Ξ,"Mar 3, 2021 0x580c05",0,0x580c05
3,Bid,0xd7510a,NaN,100.0Ξ,"Feb 21, 2021 0x5377c3",0,0x5377c3
4,Bid withdrawn,0x63a9db,NaN,69.0Ξ,"Sep 16, 2020 0xbce1fe",0,0xbce1fe
